In [24]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [25]:
print(tf.__version__)
tf.config.get_visible_devices('GPU')

2.13.0


[]

In [26]:
DATASET_SIZE = 60000
BATCH_SIZE = 128
EPOCH = 1

In [27]:
def preprocess(x, y):
    x = 2 * tf.cast(x, dtype=tf.float32) / 255. - 1.
    y = tf.cast(y, dtype=tf.int32)
    return x, y

def load():
    (train_data, train_label), (test_data, test_label) = tf.keras.datasets.mnist.load_data()
    train_db = tf.data.Dataset.from_tensor_slices((train_data, train_label))
    train_db = train_db.map(preprocess).repeat().shuffle(DATASET_SIZE).batch(batch_size=BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
    test_db = tf.data.Dataset.from_tensor_slices((test_data, test_label))
    test_db = test_db.map(preprocess).batch(batch_size=BATCH_SIZE)
    return train_db, test_db
(train_db, test_db) = load()
cnt = 0
for batch in test_db:
    if cnt >= 10:
        break
    image = batch[0][0].numpy()
    label = batch[1][0].numpy()
    print(image.shape)
    # plt.imshow(image)
    # plt.title('index: ' + str(cnt) + ' | label: ' + str(label))
    # plt.show()
    cnt = cnt + 1

(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)
(28, 28)


In [28]:
network = tf.keras.Sequential()
network.add(tf.keras.layers.Conv2D(32,3,input_shape=[28,28,1],padding='same'))#[28,28,32]
network.add(tf.keras.layers.BatchNormalization())
network.add(tf.keras.layers.ReLU())
network.add(tf.keras.layers.MaxPooling2D())#[14,14,64]
network.add(tf.keras.layers.Conv2D(64,3,padding='same'))#[14,14,64]
network.add(tf.keras.layers.BatchNormalization())
network.add(tf.keras.layers.ReLU())
network.add(tf.keras.layers.MaxPooling2D())#[7,7,127]
network.add(tf.keras.layers.Conv2D(128,3,padding='same'))#[7,7,128]
network.add(tf.keras.layers.BatchNormalization())
network.add(tf.keras.layers.ReLU())
network.add(tf.keras.layers.GlobalAveragePooling2D())
network.add(tf.keras.layers.Dense(64))
network.add(tf.keras.layers.BatchNormalization())
network.add(tf.keras.layers.ReLU())
network.add(tf.keras.layers.Dense(10,activation='softmax'))
network.build(input_shape=[None, 28, 28, 1])
network.summary()
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metrics = tf.keras.metrics.SparseCategoricalAccuracy()
network.compile(optimizer=optimizer,
              loss=loss,
              metrics=[metrics])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 batch_normalization_12 (Ba  (None, 28, 28, 32)        128       
 tchNormalization)                                               
                                                                 
 re_lu_12 (ReLU)             (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 14, 14, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 14, 14, 64)        18496     
                                                                 
 batch_normalization_13 (Ba  (None, 14, 14, 64)       

In [29]:
history = network.fit(train_db, epochs=EPOCH, steps_per_epoch=DATASET_SIZE // BATCH_SIZE, validation_data=test_db)

  4/468 [..............................] - ETA: 53s - loss: 2.0864 - sparse_categorical_accuracy: 0.3125

KeyboardInterrupt: 

In [ ]:
history_acc = history.history['sparse_categorical_accuracy']
history_val_acc = history.history['val_sparse_categorical_accuracy']
history_loss = history.history['loss']
history_val_loss = history.history['val_loss']

plt.figure()
plt.plot(history_acc, label='Training Accuracy')
plt.plot(history_val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.figure()
plt.plot(history_loss, label='Training Loss')
plt.plot(history_val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()

In [ ]:
network.evaluate(test_db, batch_size=BATCH_SIZE)

In [ ]:

network.save(r'./CNN.h5')

In [ ]:
import numpy as np
pred_y = network.predict(test_db)
result = np.argmax(pred_y, axis=1)
print(result)